In [1]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
from datetime import datetime

In [2]:
envio_bq = pd.read_csv('conferenciaHistoricoPrecos.csv', sep=',')
envio_bq['Valor'] = envio_bq['Valor'].apply(lambda x: float(x.replace('R$', '').replace('.', '').replace(',', '.')))
envio_bq['data_hora_atualizacao'] = datetime.now().strftime("%d/%m/%Y %H:%M:%S")

num_rows = envio_bq.shape[0]
num_rows

1018717

In [3]:
key_path = "leiloes-391501-477564df1a41.json"
credentials = service_account.Credentials.from_service_account_file(key_path)

In [4]:
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [5]:
envio_bq.dtypes

Valor                    float64
Marca                     object
Modelo                    object
AnoModelo                  int64
Combustivel               object
CodigoFipe                object
MesReferencia             object
Autenticacao              object
TipoVeiculo                int64
SiglaCombustivel          object
DataConsulta              object
data_hora_atualizacao     object
dtype: object

In [7]:
project_id = 'leiloes-391501'
dataset_name = 'origem_scrape_leiloes'
table_name = 'raw-historico-preco'

schema = [
    bigquery.SchemaField("Valor", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("Marca", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("Modelo", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("AnoModelo", "INT64", mode="NULLABLE"),
    bigquery.SchemaField("Combustivel", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("CodigoFipe", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("MesReferencia", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("Autenticacao", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("TipoVeiculo", "INT64", mode="NULLABLE"),
    bigquery.SchemaField("SiglaCombustivel", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("DataConsulta", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("data_hora_atualizacao", "STRING", mode="NULLABLE")
]

In [8]:
table_ref = client.dataset(dataset_name).table(table_name)
try:
    client.get_table(table_ref)
    print('A tabela já existe. Os novos dados serão adicionados a ela.')
except:
    print('A tabela não existe. Uma nova tabela será criada.')
    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table)
    print('Tabela criada com sucesso.')

A tabela não existe. Uma nova tabela será criada.
Tabela criada com sucesso.


In [9]:
job_config = bigquery.LoadJobConfig(schema=schema)
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

load_job = client.load_table_from_dataframe(envio_bq, table_ref, job_config=job_config)
print("Starting job {}".format(load_job.job_id))

load_job.result()
print("Job finished.")

destination_table = client.get_table(table_ref)
print("Loaded {} rows.".format(destination_table.num_rows))

Starting job c3432dcb-09b7-4114-8ad1-c3665b1cf692
Job finished.
Loaded 1018717 rows.


In [9]:
query = 'SELECT * FROM `leiloes-391501.origem_scrape_leiloes.raw-fipe-preco-corrente`'

dados_conferencia = client.query(query)
dados_conferencia = dados_conferencia.result().to_dataframe()

In [10]:
dados_conferencia = dados_conferencia.drop(columns=['data_hora_atualizacao'])

In [11]:
dados_conferencia.to_csv('conferenciaModelosAnosPreco.csv', index=False)

In [2]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
from datetime import datetime

key_path = "leiloes-391501-477564df1a41.json"
credentials = service_account.Credentials.from_service_account_file(key_path)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [3]:
df_consulta_total = pd.read_csv('consulta_total.csv')

C:\Users\victo\AppData\Local\Temp\ipykernel_2540\2186243507.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_consulta_total = pd.read_csv('consulta_total.csv')


In [7]:
df_consulta_total = df_consulta_total.astype(str)
df_consulta_total.dtypes

Marca                    object
cod_marca                object
cod_veiculo              object
Modelo                   object
codigoModelo             object
Ano                      object
codigoAno                object
data_hora_atualizacao    object
Codigo                   object
dtype: object

In [8]:
project_id = 'leiloes-391501'
dataset_name = 'origem_scrape_leiloes'
table_name = 'parse-dividido'

schema = [
    bigquery.SchemaField("Marca", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("cod_marca", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("cod_veiculo", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("Modelo", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("codigoModelo", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("Ano", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("codigoAno", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("data_hora_atualizacao", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("Codigo", "STRING", mode="NULLABLE"),
]


table_ref = client.dataset(dataset_name).table(table_name)
try:
    client.get_table(table_ref)
    print('A tabela já existe. Os novos dados serão adicionados a ela.')
except:
    print('A tabela não existe. Uma nova tabela será criada.')
    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table)
    print('Tabela criada com sucesso.')

A tabela já existe. Os novos dados serão adicionados a ela.


In [9]:
job_config = bigquery.LoadJobConfig(schema=schema)
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

load_job = client.load_table_from_dataframe(df_consulta_total, table_ref, job_config=job_config)
print("Starting job {}".format(load_job.job_id))

load_job.result()
print("Job finished.")

destination_table = client.get_table(table_ref)
print("Loaded {} rows.".format(destination_table.num_rows))

Starting job a1ae193e-6851-40fd-b468-88e24d4902f5
Job finished.
Loaded 8190931 rows.
